In [2]:
# Dependencies and Setup
# Dependencies
import hvplot.pandas
import pandas as pd
import requests
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")


# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("starter_code/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    xlabel = "Longitude",
    ylabel = "Latitude",
    tiles = "OSM",
    scale = 1,
    color = "City",
    size = "Humidity",
    frame_width = 800,
    frame_height = 600
)

# Display the map, map_plot_1
map_plot_1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values

city_data_ideal_weather = city_data_df.loc[
    (city_data_df["Max Temp"] > 20) & (city_data_df["Max Temp"] < 29) &
    (city_data_df["Wind Speed"] < 4.5) & (city_data_df['Cloudiness'] == 0),]

# Drop any rows with null values
no_null = city_data_ideal_weather.dropna(how='any')

# Display sample data
# YOUR CODE HERE
                               
city_data_ideal_weather



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
45,45,kapaa,22.0752,-159.3190,22.99,84,0,3.60,US,1666108257
51,51,hilo,19.7297,-155.0900,26.27,83,0,2.57,US,1666108260
63,63,banda,25.4833,80.3333,24.62,52,0,2.68,IN,1666108268
81,81,makakilo city,21.3469,-158.0858,21.66,81,0,2.57,US,1666108282
152,152,kahului,20.8947,-156.4700,23.80,60,0,3.09,US,1666108246
197,197,gat,31.6100,34.7642,24.38,100,0,3.69,IL,1666108356
211,211,laguna,38.4210,-121.4238,21.67,79,0,2.06,US,1666108364
240,240,tikaitnagar,26.9500,81.5833,23.56,59,0,0.35,IN,1666108378
265,265,san quintin,30.4833,-115.9500,21.20,74,0,1.37,MX,1666108394
340,340,santa rosalia,27.3167,-112.2833,24.62,56,0,0.74,MX,1666108436


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = city_data_ideal_weather[['City','Country', 'Lng', 'Lat', 'Humidity']].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df["Hotel Name"] = '' 

# Display sample data
hotel_df

,City,Country,Lng,Lat,Humidity,Hotel Name
45,kapaa,US,-159.3190,22.0752,84,
51,hilo,US,-155.0900,19.7297,83,
63,banda,IN,80.3333,25.4833,52,
81,makakilo city,US,-158.0858,21.3469,81,
152,kahului,US,-156.4700,20.8947,60,
197,gat,IL,34.7642,31.6100,100,
211,laguna,US,-121.4238,38.4210,79,
240,tikaitnagar,IN,81.5833,26.9500,59,
265,san quintin,MX,-115.9500,30.4833,74,
340,santa rosalia,MX,-112.2833,27.3167,56,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel

categories = "accommodation.hotel"
limit = 20

radius = 10000
params = { 
    
    "categories" : categories,
    "limit":limit,
    "radius" :radius,
    "sort" : "distance",
    "apiKey" :geoapify_key
}


# Set the parameters for the type of search


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params['filters'] = f"circle:{radius}@{longitude},{latitude}"
    params['bias'] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params).json()
    
    # Convert the API response to JSON format
    name_address = response
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kapaa - nearest hotel: Pono Kai Resort
hilo - nearest hotel: Dolphin Bay Hotel
banda - nearest hotel: #acnindiafy21
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
kahului - nearest hotel: Maui Seaside Hotel
gat - nearest hotel: מלון גולדן טאוור
laguna - nearest hotel: Holiday Inn Express & Suites
tikaitnagar - nearest hotel: Maa patesawri sweet hous
san quintin - nearest hotel: Jardines Hotel
santa rosalia - nearest hotel: Hotel del Real
narwar - nearest hotel: Shivpuri Hotel
port hedland - nearest hotel: The Esplanade Hotel
roebourne - nearest hotel: Ibis Styles Karratha
saint-francois - nearest hotel: Chez Lily
capoterra - nearest hotel: Rosa Hotel
stolac - nearest hotel: Bregava
lakki marwat - nearest hotel: Madina Hotel
san joaquin - nearest hotel: I-5 Inn
guerrero negro - nearest hotel: Plaza sal paraiso
pali - nearest hotel: Hotel Pawan International


,City,Country,Lng,Lat,Humidity,Hotel Name
45,kapaa,US,-159.3190,22.0752,84,Pono Kai Resort
51,hilo,US,-155.0900,19.7297,83,Dolphin Bay Hotel
63,banda,IN,80.3333,25.4833,52,#acnindiafy21
81,makakilo city,US,-158.0858,21.3469,81,Embassy Suites by Hilton Oahu Kapolei
152,kahului,US,-156.4700,20.8947,60,Maui Seaside Hotel
197,gat,IL,34.7642,31.6100,100,מלון גולדן טאוור
211,laguna,US,-121.4238,38.4210,79,Holiday Inn Express & Suites
240,tikaitnagar,IN,81.5833,26.9500,59,Maa patesawri sweet hous
265,san quintin,MX,-115.9500,30.4833,74,Jardines Hotel
340,santa rosalia,MX,-112.2833,27.3167,56,Hotel del Real


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# hover_color = ["City", "Country", "Hotel Name"]
# hover_line_cap = [(col, "@"+col) for col in hover_color]

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    title = "City Humidities",
    xlabel = "Longitude",
    ylabel = "Latitude",
    geo = True,
    tiles = "OSM",
    scale = 1,
    frame_width = 800,
    frame_height = 400,
    size = "Humidity",
    color = "City",
    hover_cols = ["City", "Country", "Hotel Name"]
    
)

#color the dots, also opacity & transparency
# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)